In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

print("REPLICAS: ", strategy.num_replicas_in_sync)

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU

# load the datasets
img_dir = '/kaggle/input/ussimandsegm/abdominal_US/abdominal_US/AUS/images/train'
mask_dir = '/kaggle/input/ussimandsegm/abdominal_US/abdominal_US/AUS/annotations/train'

# Load and preprocess the training images.
img_names = os.listdir(img_dir)
imgs = [Image.open(os.path.join(img_dir, img_name)) for img_name in img_names]
X = np.array([np.array(img.resize((464, 464))) for img in imgs])
X = X / 255.0  # Normalize the images to [0, 1] range.

 # Define a mapping from RGB values to class labels.
colors_to_labels = {
        (255, 255, 0): 0,  # Yellow
        (255, 0, 255): 1,  # Red
        (100, 0, 100): 2,  # Darker red
        (0, 255, 0): 3  # Green
    }

# Load, preprocess, and encode the masks.
mask_names = os.listdir(mask_dir)
masks = [Image.open(os.path.join(mask_dir, mask_name)) for mask_name in mask_names]
masks = [mask.resize((464, 464)) for mask in masks] 
Y = np.empty((len(masks), 464, 464, len(colors_to_labels)))

for i, mask in enumerate(masks):
    mask_array = np.array(mask)
    for j, (color, label) in enumerate(colors_to_labels.items()):
        Y[i, :, :, j] = np.all(mask_array == np.array(color).reshape(1, 1, 3), axis=2)

# Split the data into a training set and a validation set.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the U-Net model.
inputs = Input((464, 464, 3))
conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
conv1 = BatchNormalization()(conv1)
conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
conv1 = BatchNormalization()(conv1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
conv2 = BatchNormalization()(conv2)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
conv2 = BatchNormalization()(conv2)
drop2 = Dropout(0.5)(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
conv3 = BatchNormalization()(conv3)
conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
conv3 = BatchNormalization()(conv3)
drop3 = Dropout(0.5)(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
conv4 = BatchNormalization()(conv4)
conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
conv4 = BatchNormalization()(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    
conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
conv5 = BatchNormalization()(conv5)
conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
conv5 = BatchNormalization()(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(drop5)
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(256, 3, activation='relu', padding='same')(merge6)
conv6 = BatchNormalization()(conv6)
conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)
conv6 = BatchNormalization()(conv6)
drop6= Dropout(0.5)(conv6)

up7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(drop6)
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
conv7 = BatchNormalization()(conv7)
conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)
conv7 = BatchNormalization()(conv7)
drop7= Dropout(0.5)(conv7)

up8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(drop7)
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
conv8 = BatchNormalization()(conv8)
conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)
conv8 = BatchNormalization()(conv8)
drop8= Dropout(0.5)(conv8)

up9 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(drop8)
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
conv9 = BatchNormalization()(conv9)
conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
conv9 = BatchNormalization()(conv9)

conv10 = Conv2D(len(colors_to_labels), 1, activation='softmax')(conv9)
    
    
model = Model(inputs, conv10)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy')


# Train the model.
results = model.fit(X_train, Y_train, batch_size=4, epochs=15, validation_data=(X_val, Y_val))

D0617 21:45:18.704755936      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0617 21:45:18.704781240      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0617 21:45:18.704784612      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0617 21:45:18.704787100      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0617 21:45:18.704789076      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0617 21:45:18.704791472      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0617 21:45:18.704793777      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0617 21:45:18.

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8
Epoch 1/15
127/127 [==============================] - 501s 4s/step - loss: 0.1960 - val_loss: 0.1562
Epoch 2/15
127/127 [==============================] - 487s 4s/step - loss: 0.1490 - val_loss: 0.1279
Epoch 3/15
127/127 [==============================] - 488s 4s/step - loss: 0.1458 - val_loss: 0.1429
Epoch 4/15
127/127 [==============================] - 490s 4s/step - loss: 0.1479 - val_loss: 0.1184
Epoch 5/15
127/127 [==============================] - 487s 4s/step - loss: 0.1443 - val_loss: 0.1043
Epoch 6/15
127/127 [==============================] - 490s 4s/step - loss: 0.1425 - val_loss: 0.1105
Epoch 7/15
127/127 [==============================] - 491s 4s/step - loss: 0.1340 - val_loss: 0.1285
Epoch 8/15
127/127 [==============================] - 488s 4s/step - loss: 0.1310 - val_loss: 0.1290
Epoch 9/15
127/127 [==============================] - 488s 4s/step - loss: 0.1309 - val_loss: 0.1136
Epoch 10/15
127/127 [==============================] - 484s 4s/step - loss: 0.

In [2]:
import cv2
import os
import numpy as np

# Define path
path_to_test = '/kaggle/input/ussimandsegm/abdominal_US/abdominal_US/AUS/images/test'

# Load and resize images from directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img_resized = cv2.resize(img, (464, 464)) # Resize image to expected dimensions
            images.append(img_resized)
    return images

# Using the function to load and resize test images
X_test = load_images_from_folder(path_to_test)

# Convert list to numpy array
X_test = np.array(X_test)

# Normalize the test data like we did with train data
X_test = X_test / 255.0

# Use model to make predictions
predictions = model.predict(X_test)


10/10 [==============================] - 36s 4s/step
